# Explore, segment, and cluster the neighborhoods in the city of Toronto

In this assignment, you will be required to explore, segment, and cluster the neighborhoods in the city of Toronto. However, unlike New York, the neighborhood data is not readily available on the internet. What is interesting about the field of data science is that each project can be challenging in its unique way, so you need to learn to be agile and refine the skill to learn new libraries and tools quickly depending on the project.

For the Toronto neighborhood data, a Wikipedia page exists that has all the information we need to explore and cluster the neighborhoods in Toronto. You will be required to scrape the Wikipedia page and wrangle the data, clean it, and then read it into a pandas dataframe so that it is in a structured format like the New York dataset.

Once the data is in a structured format, you can replicate the analysis that we did to the New York City dataset to explore and cluster the neighborhoods in the city of Toronto.

Your submission will be a link to your Jupyter Notebook on your Github repository.

## Get Toronto neighborhood data

For this assignment, you will be required to explore and cluster the neighborhoods in Toronto.

1. Start by creating a new Notebook for this assignment.
2. Use the Notebook to build the code to scrape the following Wikipedia page, https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M, in order to obtain the data that is in the table of postal codes and to transform the data into a pandas dataframe like the one shown below:
![](https://d3c33hcgiwev3.cloudfront.net/imageAssetProxy.v1/7JXaz3NNEeiMwApe4i-fLg_40e690ae0e927abda2d4bde7d94ed133_Screen-Shot-2018-06-18-at-7.17.57-PM.png?expiry=1566000000000&hmac=AN3CJ7qeqs6bod-Dt7oM7fnL3e5Hx_ERYwMV3M1TSyM)

3. To create the above dataframe:

- The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood
- Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.
- More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.
- If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough. So for the 9th cell in the table on the Wikipedia page, the value of the Borough and the Neighborhood columns will be Queen's Park.
- Clean your Notebook and add Markdown cells to explain your work and any assumptions you are making.
- In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.
4. Submit a link to your Notebook on your Github repository.

### Pull the data

Let's load the necessary libraries first...

In [21]:
### libraries
import requests
import pandas as pd

#!conda install -c conda-forge beautifulsoup4 lxml html5lib --yes
from bs4 import BeautifulSoup

Now let's pull down the data from that Wiki page...

In [22]:
### data
wiki_url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

# request
myres = requests.get(wiki_url)
mysoup = BeautifulSoup(myres.content, 'html5lib')

# read table into df
mytable = mysoup.find_all('table')[0] 
mydf = pd.read_html(str(mytable))[0]

# check it
mydf.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


### Process the data

Time for some filtering and postprocessing...

In [25]:
# get rid of the "Not assigned" boroughs
mydf_bor = mydf[~mydf.Borough.isin(['Not assigned'])]
mydf_bor.head()

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights


Postcodes appearing multiple times should be combined into one line with the neighbourhoods concatenated...

In [34]:
# make those postcodes unique
mydf_combined = mydf_bor.groupby(['Postcode','Borough'])['Neighbourhood'].apply(', '.join).reset_index()
mydf_combined.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


Finally, let's fix those rows where the Neighbourhood is "not assigned" by filling it with the content of the Borough column...

In [33]:
# fix the neighbourhood column where it contains "not assigned"
mydf_combined.loc[mydf_combined['Neighbourhood']=="Not assigned", 'Neighbourhood'] = mydf_combined.loc[mydf_combined['Neighbourhood']=="Not assigned", 'Borough']
mydf_combined.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [38]:
# save the Toronto data to CSV for later use
mydf_combined.to_csv('toronto_postcodes.csv', index=False)

In [35]:
mydf_combined.shape

(103, 3)